In [2]:
import numpy as np
import cv2
import timm
import tqdm

import torch
from torchvision import datasets, transforms
from torch.utils import data

from sklearn.metrics import accuracy_score, balanced_accuracy_score, classification_report, confusion_matrix
from PIL import Image

from matplotlib import pyplot as plt


np.random.seed(42)
NUM_CLASSES = 9
BATCH_SIZE = 32
device = torch.device("cuda")
PATH_TO_TEST_DATASET = "data/CRC-VAL-HE-7K/"


def cv2_loader(path: str):
    return Image.fromarray(cv2.imread(path, -1)[:, :, ::-1])


/home/user/anaconda3/lib/python3.9/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: '/home/user/anaconda3/lib/python3.9/site-packages/torchvision/image.so: undefined symbol: _ZN3c1017RegisterOperatorsD1Ev'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(
2024-07-20 00:56:08.391350: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/user/anaconda3/lib/python3.9/site-packages/cv2/../../lib64:
2024-07-20 00:56:08.391365: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [3]:
model = timm.create_model('efficientnet_b1.ft_in1k', pretrained=True, num_classes=NUM_CLASSES, in_chans=3, global_pool='avg')
model.load_state_dict(torch.load('./0.9774_0.9688.pt'))
model.to('cuda')
model.eval()

Layer (type:depth-idx)                        Output Shape              Param #
EfficientNet                                  [1, 9]                    --
├─Conv2d: 1-1                                 [1, 32, 112, 112]         864
├─BatchNormAct2d: 1-2                         [1, 32, 112, 112]         64
│    └─Identity: 2-1                          [1, 32, 112, 112]         --
│    └─SiLU: 2-2                              [1, 32, 112, 112]         --
├─Sequential: 1-3                             [1, 320, 7, 7]            --
│    └─Sequential: 2-3                        [1, 16, 112, 112]         --
│    │    └─DepthwiseSeparableConv: 3-1       [1, 16, 112, 112]         1,448
│    │    └─DepthwiseSeparableConv: 3-2       [1, 16, 112, 112]         612
│    └─Sequential: 2-4                        [1, 24, 56, 56]           --
│    │    └─InvertedResidual: 3-3             [1, 24, 56, 56]           6,004
│    │    └─InvertedResidual: 3-4             [1, 24, 56, 56]           10,710
│    │  

In [14]:
def evaluate(model, test_dataloader):
    print("Running Evaluation...")

    targets_array = []
    predictions_array = []

    with torch.no_grad():

        test_iter = iter(test_dataloader)
        for j in tqdm.tqdm(range(len(test_dataloader))):

            image, labels = next(test_iter)
            image = image.to(device, non_blocking=True)
            labels = labels.to(device, non_blocking=True)


            predictions = model(image)
            predictions2 = model(image.flip(2))
            predictions3 = model(image.flip(3))
            predictions23 = model(image.flip(2).flip(3))
            
            _, predictions = torch.max(predictions.data + predictions2.data + predictions3.data  + predictions23.data, 1)

            predictions = predictions.detach().cpu().numpy()
            targets = labels.detach().cpu().numpy()

            for k in range(targets.shape[0]):

                target = targets[k]
                predicted = predictions[k]

                targets_array.append(target)
                predictions_array.append(predicted)

        print("Accuracy: " + str(accuracy_score(targets_array, predictions_array)))
        print("Balanced Accuracy: " + str(balanced_accuracy_score(targets_array, predictions_array)))
        

        print(classification_report(targets_array, predictions_array, digits=3))
        print(confusion_matrix(targets_array, predictions_array))
        
        
        return predictions_array

In [15]:
test_transforms = transforms.Compose([transforms.ToTensor(), transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])
test_dataset = datasets.ImageFolder(PATH_TO_TEST_DATASET, transform=test_transforms, loader=cv2_loader)
test_dataloader = data.DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=4, pin_memory=False, drop_last=False)

predictions_no_hue_aug = evaluate(model, test_dataloader)

Running Evaluation...


100%|████████████████████████████████████████████████████████████████████████████████████████| 225/225 [01:00<00:00,  3.73it/s]

Accuracy: 0.9772980501392757
Balanced Accuracy: 0.9679702454961778
              precision    recall  f1-score   support

           0      0.996     0.993     0.994      1338
           1      0.999     1.000     0.999       847
           2      0.960     0.982     0.971       339
           3      0.975     0.997     0.986       634
           4      0.997     0.990     0.994      1035
           5      0.880     0.992     0.932       592
           6      0.989     0.976     0.982       741
           7      0.955     0.808     0.875       421
           8      0.984     0.975     0.980      1233

    accuracy                          0.977      7180
   macro avg      0.971     0.968     0.968      7180
weighted avg      0.978     0.977     0.977      7180

[[1328    1    0    0    0    9    0    0    0]
 [   0  847    0    0    0    0    0    0    0]
 [   0    0  333    0    0    1    0    5    0]
 [   0    0    1  632    0    0    0    1    0]
 [   6    0    0    0 1025    0    0